<a href="https://colab.research.google.com/github/ShashankShorya0211/MIMDPU/blob/main/06-110924.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import re
from collections import namedtuple
from enum import Enum

import re
from enum import Enum

class TokenType(Enum):
    NUMBER = 'NUMBER'
    KEYWORD = 'KEYWORD'
    IDENTIFIER = 'IDENTIFIER'
    OPERATOR = 'OPERATOR'
    PUNCTUATOR = 'PUNCTUATOR'
    EOF = 'EOF'

class Token:
    def __init__(self, type, value, line, column):
        self.type = type
        self.value = value
        self.line = line
        self.column = column

    def __str__(self):
        return f'Token({self.type}, {self.value}, line={self.line}, col={self.column})'

class Lexer:
    def __init__(self, source_code):
        self.source_code = source_code
        self.position = 0
        self.line = 1
        self.column = 1
        self.current_char = self.source_code[self.position] if self.position < len(self.source_code) else None

    def advance(self):
        if self.current_char == '\n':
            self.line += 1
            self.column = 1
        else:
            self.column += 1
        self.position += 1
        self.current_char = self.source_code[self.position] if self.position < len(self.source_code) else None

    def skip_whitespace(self):
        while self.current_char is not None and self.current_char.isspace():
            self.advance()

    def get_number(self):
        result = ''
        while self.current_char is not None and self.current_char.isdigit():
            result += self.current_char
            self.advance()
        if self.current_char == '.':
            result += self.current_char
            self.advance()
            while self.current_char is not None and self.current_char.isdigit():
                result += self.current_char
                self.advance()
        return float(result) if '.' in result else int(result)

    def get_identifier(self):
        result = ''
        while self.current_char is not None and (self.current_char.isalnum() or self.current_char == '_'):
            result += self.current_char
            self.advance()
        return result

    def tokenize(self):
        tokens = []
        keywords = {'int', 'float', 'parallel', 'if', 'else', 'while', 'for', 'return', 'void'}
        operators = {'+': 'PLUS', '-': 'MINUS', '*': 'MULTIPLY', '/': 'DIVIDE', '=': 'ASSIGN',
                     '==': 'EQUALS', '!=': 'NOT_EQUALS', '<': 'LESS_THAN', '>': 'GREATER_THAN',
                     '<=': 'LESS_EQUAL', '>=': 'GREATER_EQUAL', '&&': 'AND', '||': 'OR', '!': 'NOT'}
        punctuators = {'{': 'LBRACE', '}': 'RBRACE', '(': 'LPAREN', ')': 'RPAREN', ';': 'SEMI', ',': 'COMMA'}

        while self.current_char is not None:
            if self.current_char.isspace():
                self.skip_whitespace()
                continue

            if self.current_char.isdigit():
                tokens.append(Token(TokenType.NUMBER, self.get_number(), self.line, self.column))
            elif self.current_char.isalpha() or self.current_char == '_':
                identifier = self.get_identifier()
                if identifier in keywords:
                    tokens.append(Token(TokenType.KEYWORD, identifier, self.line, self.column))
                else:
                    tokens.append(Token(TokenType.IDENTIFIER, identifier, self.line, self.column))
            elif self.current_char in operators or self.current_char in punctuators:
                start_column = self.column
                op = self.current_char
                self.advance()
                if op + self.current_char in operators:
                    op += self.current_char
                    self.advance()
                if op in operators:
                    tokens.append(Token(TokenType.OPERATOR, operators[op], self.line, start_column))
                elif op in punctuators:
                    tokens.append(Token(TokenType.PUNCTUATOR, punctuators[op], self.line, start_column))
            else:
                raise Exception(f'Invalid character: {self.current_char} at line {self.line}, column {self.column}')

        tokens.append(Token(TokenType.EOF, None, self.line, self.column))
        return tokens

class ASTNode:
    def __init__(self, type, children=None, value=None):
        self.type = type
        self.children = children if children else []
        self.value = value

class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.current_token = None
        self.token_index = -1
        self.advance()

    def advance(self):
        self.token_index += 1
        if self.token_index < len(self.tokens):
            self.current_token = self.tokens[self.token_index]
        else:
            self.current_token = Token(TokenType.EOF, None, -1, -1)

    def eat(self, token_type):
        if self.current_token.type == token_type:
            token = self.current_token
            self.advance()
            return token
        else:
            raise Exception(f'Expected {token_type}, got {self.current_token.type} at line {self.current_token.line}, column {self.current_token.column}')

    def parse(self):
        return self.program()

    def program(self):
        node = ASTNode('Program')
        while self.current_token.type != TokenType.EOF:
            node.children.append(self.function_definition())
        return node

    def function_definition(self):
        return_type = self.eat(TokenType.KEYWORD).value
        name = self.eat(TokenType.IDENTIFIER).value
        self.eat(TokenType.PUNCTUATOR)  # (
        params = self.parameter_list()
        self.eat(TokenType.PUNCTUATOR)  # )
        body = self.compound_statement()
        return ASTNode('FunctionDefinition', [ASTNode('ReturnType', value=return_type), ASTNode('FunctionName', value=name), params, body])

    def parameter_list(self):
        params = []
        while self.current_token.type != TokenType.PUNCTUATOR or self.current_token.value != 'RPAREN':
            param_type = self.eat(TokenType.KEYWORD).value
            param_name = self.eat(TokenType.IDENTIFIER).value
            params.append(ASTNode('Parameter', [ASTNode('Type', value=param_type), ASTNode('Name', value=param_name)]))
            if self.current_token.type == TokenType.PUNCTUATOR and self.current_token.value == 'COMMA':
                self.advance()
        return ASTNode('ParameterList', params)

    def compound_statement(self):
        self.eat(TokenType.PUNCTUATOR)  # {
        statements = []
        while self.current_token.type != TokenType.PUNCTUATOR or self.current_token.value != 'RBRACE':
            statements.append(self.statement())
        self.eat(TokenType.PUNCTUATOR)  # }
        return ASTNode('CompoundStatement', statements)

    def statement(self):
        if self.current_token.type == TokenType.KEYWORD:
            if self.current_token.value in ['int', 'float']:
                return self.variable_declaration()
            elif self.current_token.value == 'parallel':
                return self.parallel_statement()
            elif self.current_token.value == 'if':
                return self.if_statement()
            elif self.current_token.value == 'while':
                return self.while_statement()
            elif self.current_token.value == 'for':
                return self.for_statement()
            elif self.current_token.value == 'return':
                return self.return_statement()
        elif self.current_token.type == TokenType.IDENTIFIER:
            return self.assignment_statement()
        else:
            raise Exception(f'Unexpected token: {self.current_token.type} at line {self.current_token.line}, column {self.current_token.column}')

    def variable_declaration(self):
        var_type = self.eat(TokenType.KEYWORD).value
        var_name = self.eat(TokenType.IDENTIFIER).value

        if self.current_token.type == TokenType.OPERATOR and self.current_token.value == 'ASSIGN':
            self.eat(TokenType.OPERATOR)  # Consume the '='
            init_expr = self.expression()
            self.eat(TokenType.PUNCTUATOR)  # Consume the ';'
            return ASTNode('VariableDeclaration', [
                ASTNode('Type', value=var_type),
                ASTNode('Name', value=var_name),
                init_expr
            ])
        else:
            self.eat(TokenType.PUNCTUATOR)  # Consume the ';'
            return ASTNode('VariableDeclaration', [
                ASTNode('Type', value=var_type),
                ASTNode('Name', value=var_name)
            ])

    def parallel_statement(self):
        self.eat(TokenType.KEYWORD)  # parallel
        return ASTNode('ParallelStatement', [self.compound_statement()])

    def if_statement(self):
        self.eat(TokenType.KEYWORD)  # if
        self.eat(TokenType.PUNCTUATOR)  # (
        condition = self.expression()
        self.eat(TokenType.PUNCTUATOR)  # )
        if_body = self.compound_statement()
        else_body = None
        if self.current_token.type == TokenType.KEYWORD and self.current_token.value == 'else':
            self.advance()
            else_body = self.compound_statement()
        return ASTNode('IfStatement', [condition, if_body, else_body] if else_body else [condition, if_body])

    def while_statement(self):
        self.eat(TokenType.KEYWORD)  # while
        self.eat(TokenType.PUNCTUATOR)  # (
        condition = self.expression()
        self.eat(TokenType.PUNCTUATOR)  # )
        body = self.compound_statement()
        return ASTNode('WhileStatement', [condition, body])

    def for_statement(self):
        self.eat(TokenType.KEYWORD)  # for
        self.eat(TokenType.PUNCTUATOR)  # (
        init = self.statement()
        condition = self.expression()
        self.eat(TokenType.PUNCTUATOR)  # ;
        update = self.expression()
        self.eat(TokenType.PUNCTUATOR)  # )
        body = self.compound_statement()
        return ASTNode('ForStatement', [init, condition, update, body])

    def return_statement(self):
        self.eat(TokenType.KEYWORD)  # return
        expr = self.expression()
        self.eat(TokenType.PUNCTUATOR)  # ;
        return ASTNode('ReturnStatement', [expr])

    def assignment_statement(self):
        var_name = self.eat(TokenType.IDENTIFIER).value
        self.eat(TokenType.OPERATOR)  # =
        expr = self.expression()
        self.eat(TokenType.PUNCTUATOR)  # ;
        return ASTNode('Assignment', [ASTNode('Variable', value=var_name), expr])

    def expression(self):
        return self.logical_or()

    def logical_or(self):
        node = self.logical_and()
        while self.current_token.type == TokenType.OPERATOR and self.current_token.value == 'OR':
            op = self.eat(TokenType.OPERATOR).value
            node = ASTNode('BinaryOp', [node, self.logical_and()], value=op)
        return node

    def logical_and(self):
        node = self.equality()
        while self.current_token.type == TokenType.OPERATOR and self.current_token.value == 'AND':
            op = self.eat(TokenType.OPERATOR).value
            node = ASTNode('BinaryOp', [node, self.equality()], value=op)
        return node

    def equality(self):
        node = self.relational()
        while self.current_token.type == TokenType.OPERATOR and self.current_token.value in ['EQUALS', 'NOT_EQUALS']:
            op = self.eat(TokenType.OPERATOR).value
            node = ASTNode('BinaryOp', [node, self.relational()], value=op)
        return node

    def relational(self):
        node = self.additive()
        while self.current_token.type == TokenType.OPERATOR and self.current_token.value in ['LESS_THAN', 'GREATER_THAN', 'LESS_EQUAL', 'GREATER_EQUAL']:
            op = self.eat(TokenType.OPERATOR).value
            node = ASTNode('BinaryOp', [node, self.additive()], value=op)
        return node

    def additive(self):
        node = self.multiplicative()
        while self.current_token.type == TokenType.OPERATOR and self.current_token.value in ['PLUS', 'MINUS']:
            op = self.eat(TokenType.OPERATOR).value
            node = ASTNode('BinaryOp', [node, self.multiplicative()], value=op)
        return node

    def multiplicative(self):
        node = self.unary()
        while self.current_token.type == TokenType.OPERATOR and self.current_token.value in ['MULTIPLY', 'DIVIDE']:
            op = self.eat(TokenType.OPERATOR).value
            node = ASTNode('BinaryOp', [node, self.unary()], value=op)
        return node

    def unary(self):
        if self.current_token.type == TokenType.OPERATOR and self.current_token.value in ['PLUS', 'MINUS', 'NOT']:
            op = self.eat(TokenType.OPERATOR).value
            return ASTNode('UnaryOp', [self.unary()], value=op)
        return self.primary()

    def primary(self):
        token = self.current_token
        if token.type == TokenType.NUMBER:
            self.advance()
            return ASTNode('Literal', value=token.value)
        elif token.type == TokenType.IDENTIFIER:
            self.advance()
            return ASTNode('Variable', value=token.value)
        elif token.type == TokenType.PUNCTUATOR and token.value == 'LPAREN':
            self.advance()
            expr = self.expression()
            self.eat(TokenType.PUNCTUATOR)  # )
            return expr
        else:
            raise Exception(f'Unexpected token: {token.type} at line {token.line}, column {token.column}')

class SymbolTable:
    def __init__(self):
        self.symbols = {}
        self.parent = None

    def define(self, name, type):
        self.symbols[name] = type

    def lookup(self, name):
        if name in self.symbols:
            return self.symbols[name]
        elif self.parent:
            return self.parent.lookup(name)
        else:
            return None

class SemanticAnalyzer:
    def __init__(self, ast):
        self.ast = ast
        self.current_scope = SymbolTable()
        self.function_table = {}

    def analyze(self):
        self.visit(self.ast)

    def visit(self, node):
        method_name = f'visit_{node.type}'
        method = getattr(self, method_name, self.generic_visit)
        return method(node)

    def generic_visit(self, node):
        for child in node.children:
            self.visit(child)

    def visit_Program(self, node):
        for child in node.children:
            self.visit(child)

    def visit_FunctionDefinition(self, node):
        return_type = node.children[0].value
        function_name = node.children[1].value
        params = node.children[2]
        body = node.children[3]

        if function_name in self.function_table:
            raise Exception(f"Function {function_name} already defined")

        self.function_table[function_name] = {
            'return_type': return_type,
            'params': self.visit(params)
        }

        # Create a new scope for the function
        function_scope = SymbolTable()
        function_scope.parent = self.current_scope
        self.current_scope = function_scope

        # Add parameters to the function scope
        for param in self.function_table[function_name]['params']:
            self.current_scope.define(param['name'], param['type'])

        self.visit(body)

        # Restore the previous scope
        self.current_scope = self.current_scope.parent

    def visit_ParameterList(self, node):
        params = []
        for param in node.children:
            param_type = param.children[0].value
            param_name = param.children[1].value
            params.append({'name': param_name, 'type': param_type})
        return params

    def visit_VariableDeclaration(self, node):
        var_type = node.children[0].value
        var_name = node.children[1].value
        if self.current_scope.lookup(var_name):
            raise Exception(f"Variable {var_name} already declared in this scope")
        self.current_scope.define(var_name, var_type)

        if len(node.children) > 2:
            init_expr_type = self.visit(node.children[2])
            if init_expr_type != var_type:
                raise Exception(f"Type mismatch in initialization of {var_name}: expected {var_type}, got {init_expr_type}")


    def visit_Assignment(self, node):
        var_name = node.children[0].value
        expr_type = self.visit(node.children[1])
        var_type = self.current_scope.lookup(var_name)
        if not var_type:
            raise Exception(f"Variable {var_name} not declared")
        if var_type != expr_type:
            raise Exception(f"Type mismatch in assignment to {var_name}: expected {var_type}, got {expr_type}")

    def visit_ParallelStatement(self, node):
        # Create a new scope for the parallel block
        parallel_scope = SymbolTable()
        parallel_scope.parent = self.current_scope
        self.current_scope = parallel_scope

        self.visit(node.children[0])  # Visit the compound statement

        # Restore the previous scope
        self.current_scope = self.current_scope.parent

    def visit_IfStatement(self, node):
        condition_type = self.visit(node.children[0])
        if condition_type != 'bool':
            raise Exception("Condition in if statement must be of type bool")
        self.visit(node.children[1])  # if body
        if len(node.children) > 2:
            self.visit(node.children[2])  # else body

    def visit_WhileStatement(self, node):
        condition_type = self.visit(node.children[0])
        if condition_type != 'bool':
            raise Exception("Condition in while statement must be of type bool")
        self.visit(node.children[1])  # body

    def visit_ForStatement(self, node):
        # Create a new scope for the for loop
        loop_scope = SymbolTable()
        loop_scope.parent = self.current_scope
        self.current_scope = loop_scope

        self.visit(node.children[0])  # initialization
        condition_type = self.visit(node.children[1])
        if condition_type != 'bool':
            raise Exception("Condition in for statement must be of type bool")
        self.visit(node.children[2])  # update
        self.visit(node.children[3])  # body

        # Restore the previous scope
        self.current_scope = self.current_scope.parent

    def visit_ReturnStatement(self, node):
        expr_type = self.visit(node.children[0])
        # Check if the return type matches the function's declared return type
        # This would require keeping track of the current function, which is not implemented in this simple version

    def visit_BinaryOp(self, node):
        left_type = self.visit(node.children[0])
        right_type = self.visit(node.children[1])
        if left_type != right_type:
            raise Exception(f"Type mismatch in binary operation: {left_type} {node.value} {right_type}")
        if node.value in ['PLUS', 'MINUS', 'MULTIPLY', 'DIVIDE']:
            return left_type
        elif node.value in ['LESS_THAN', 'GREATER_THAN', 'LESS_EQUAL', 'GREATER_EQUAL', 'EQUALS', 'NOT_EQUALS', 'AND', 'OR']:
            return 'bool'
        else:
            raise Exception(f"Unknown binary operator: {node.value}")

    def visit_UnaryOp(self, node):
        operand_type = self.visit(node.children[0])
        if node.value in ['PLUS', 'MINUS']:
            if operand_type not in ['int', 'float']:
                raise Exception(f"Unary {node.value} can only be applied to int or float")
            return operand_type
        elif node.value == 'NOT':
            if operand_type != 'bool':
                raise Exception("Unary NOT can only be applied to bool")
            return 'bool'
        else:
            raise Exception(f"Unknown unary operator: {node.value}")

    def visit_Literal(self, node):
        if isinstance(node.value, int):
            return 'int'
        elif isinstance(node.value, float):
            return 'float'
        elif isinstance(node.value, bool):
            return 'bool'
        else:
            raise Exception(f"Unknown literal type: {type(node.value)}")

    def visit_Variable(self, node):
        var_type = self.current_scope.lookup(node.value)
        if not var_type:
            raise Exception(f"Variable {node.value} not declared")
        return var_type

class IRInstruction:
    def __init__(self, op, args):
        self.op = op
        self.args = args

    def __str__(self):
        return f"{self.op} {' '.join(map(str, self.args))}"

class IRGenerator:
    def __init__(self, ast):
        self.ast = ast
        self.instructions = []
        self.temp_counter = 0
        self.label_counter = 0
        self.current_function = None
        self.parallel_depth = 0

    def generate(self):
        self.visit(self.ast)
        return self.instructions

    def emit(self, op, args):
        self.instructions.append(IRInstruction(op, args))

    def visit(self, node):
        method_name = f'visit_{node.type}'
        method = getattr(self, method_name, self.generic_visit)
        return method(node)

    def generic_visit(self, node):
        raise Exception(f'No visit_{node.type} method')

    def visit_CompoundStatement(self, node):
        for child in node.children:
            self.visit(child)

    def visit_Program(self, node):
        for child in node.children:
            self.visit(child)

    def visit_FunctionDefinition(self, node):
        function_name = node.children[1].value
        self.current_function = function_name
        self.emit('FUNCTION_BEGIN', [function_name])
        self.visit(node.children[3])  # Visit function body
        self.emit('FUNCTION_END', [function_name])
        self.current_function = None

    def visit_VariableDeclaration(self, node):
        var_name = node.children[1].value
        self.emit('ALLOC', [var_name])

        if len(node.children) > 2:
            init_value = self.visit(node.children[2])
            self.emit('STORE', [var_name, init_value])

    def visit_Assignment(self, node):
        var_name = node.children[0].value
        value = self.visit(node.children[1])
        self.emit('STORE', [var_name, value])

    def visit_ParallelStatement(self, node):
        self.parallel_depth += 1
        self.emit('PARALLEL_BEGIN', [])
        self.visit(node.children[0])  # Visit the compound statement
        self.emit('PARALLEL_END', [])
        self.parallel_depth -= 1

    def visit_IfStatement(self, node):
        condition = self.visit(node.children[0])
        false_label = self.new_label()
        end_label = self.new_label()

        self.emit('JUMP_IF_FALSE', [condition, false_label])
        self.visit(node.children[1])  # if body
        self.emit('JUMP', [end_label])
        self.emit('LABEL', [false_label])
        if len(node.children) > 2:
            self.visit(node.children[2])  # else body
        self.emit('LABEL', [end_label])

    def visit_WhileStatement(self, node):
        start_label = self.new_label()
        end_label = self.new_label()

        self.emit('LABEL', [start_label])
        condition = self.visit(node.children[0])
        self.emit('JUMP_IF_FALSE', [condition, end_label])
        self.visit(node.children[1])  # body
        self.emit('JUMP', [start_label])
        self.emit('LABEL', [end_label])

    def visit_ForStatement(self, node):
        self.visit(node.children[0])  # initialization
        start_label = self.new_label()
        end_label = self.new_label()

        self.emit('LABEL', [start_label])
        condition = self.visit(node.children[1])
        self.emit('JUMP_IF_FALSE', [condition, end_label])
        self.visit(node.children[3])  # body
        self.visit(node.children[2])  # update
        self.emit('JUMP', [start_label])
        self.emit('LABEL', [end_label])

    def visit_ReturnStatement(self, node):
        value = self.visit(node.children[0])
        self.emit('RETURN', [value])

    def visit_BinaryOp(self, node):
        left = self.visit(node.children[0])
        right = self.visit(node.children[1])
        result = self.new_temp()
        operation = {
            'PLUS': 'ADD',
            'MINUS': 'SUB',
            'MULTIPLY': 'MUL',
            'DIVIDE': 'DIV',
            'LESS_THAN': 'LT',
            'GREATER_THAN': 'GT',
            'LESS_EQUAL': 'LE',
            'GREATER_EQUAL': 'GE',
            'EQUALS': 'EQ',
            'NOT_EQUALS': 'NE',
            'AND': 'AND',
            'OR': 'OR'
        }.get(node.value, node.value)
        self.emit(operation, [result, left, right])
        return result

    def visit_UnaryOp(self, node):
        operand = self.visit(node.children[0])
        result = self.new_temp()
        operation = {
            'PLUS': 'POS',
            'MINUS': 'NEG',
            'NOT': 'NOT'
        }.get(node.value, node.value)
        self.emit(operation, [result, operand])
        return result

    def visit_Literal(self, node):
        result = self.new_temp()
        self.emit('LOAD_CONST', [result, node.value])
        return result

    def visit_Variable(self, node):
        result = self.new_temp()
        self.emit('LOAD', [result, node.value])
        return result

    def emit(self, op, args):
        instruction = IRInstruction(op, args)
        self.instructions.append(instruction)

    def new_temp(self):
        self.temp_counter += 1
        return f't{self.temp_counter}'

    def new_label(self):
        self.label_counter += 1
        return f'L{self.label_counter}'

class IROptimizer:
    def __init__(self, ir):
        self.ir = ir

    def optimize(self):
        self.constant_folding()
        self.dead_code_elimination()
        self.common_subexpression_elimination()
        self.optimize_parallel_blocks()
        return self.ir

    def constant_folding(self):
        constants = {}
        new_ir = []
        for inst in self.ir:
            if inst.op == 'LOAD_CONST':
                constants[inst.args[0]] = inst.args[1]
            elif inst.op in ['ADD', 'SUB', 'MUL', 'DIV', 'AND', 'OR']:
                result, op1, op2 = inst.args
                if op1 in constants and op2 in constants:
                    value = self.compute_constant(inst.op, constants[op1], constants[op2])
                    new_ir.append(IRInstruction('LOAD_CONST', [result, value]))
                    constants[result] = value
                else:
                    new_ir.append(inst)
                    if inst.op != 'STORE':
                        constants.pop(result, None)
            else:
                new_ir.append(inst)
                if inst.op != 'STORE':
                    for arg in inst.args:
                        constants.pop(arg, None)
        self.ir = new_ir

    def compute_constant(self, op, val1, val2):
        if op == 'ADD':
            return val1 + val2
        elif op == 'SUB':
            return val1 - val2
        elif op == 'MUL':
            return val1 * val2
        elif op == 'DIV':
            return val1 // val2 if isinstance(val1, int) and isinstance(val2, int) else val1 / val2
        elif op == 'AND':
            return val1 and val2
        elif op == 'OR':
            return val1 or val2

    def dead_code_elimination(self):
        used_vars = set()
        new_ir = []
        for inst in reversed(self.ir):
            if inst.op in ['STORE', 'RETURN'] or inst.op.startswith('JUMP'):
                new_ir.insert(0, inst)
                used_vars.update(inst.args)
            elif any(arg in used_vars for arg in inst.args):
                new_ir.insert(0, inst)
                used_vars.update(inst.args[1:])
                used_vars.add(inst.args[0])
        self.ir = new_ir

    def common_subexpression_elimination(self):
        expr_to_var = {}
        new_ir = []
        for inst in self.ir:
            if inst.op in ['ADD', 'SUB', 'MUL', 'DIV', 'AND', 'OR']:
                expr = (inst.op, inst.args[1], inst.args[2])
                if expr in expr_to_var:
                    new_ir.append(IRInstruction('MOVE', [inst.args[0], expr_to_var[expr]]))
                else:
                    new_ir.append(inst)
                    expr_to_var[expr] = inst.args[0]
            else:
                new_ir.append(inst)
                if inst.op == 'STORE':
                    expr_to_var = {k: v for k, v in expr_to_var.items() if v != inst.args[0]}
        self.ir = new_ir

    def optimize_parallel_blocks(self):
        new_ir = []
        parallel_block = []
        in_parallel = False
        for inst in self.ir:
            if inst.op == 'PARALLEL_BEGIN':
                in_parallel = True
                parallel_block = []
            elif inst.op == 'PARALLEL_END':
                in_parallel = False
                optimized_block = self.balance_parallel_block(parallel_block)
                new_ir.extend(optimized_block)
                new_ir.append(inst)
            elif in_parallel:
                parallel_block.append(inst)
            else:
                new_ir.append(inst)
        self.ir = new_ir

    def balance_parallel_block(self, block):
        # This is a simplified load balancing strategy.
        # In a real compiler, this would be much more sophisticated.
        num_processors = 4  # Assuming 4 processors for MIMD
        balanced_blocks = [[] for _ in range(num_processors)]
        for i, inst in enumerate(block):
            balanced_blocks[i % num_processors].append(inst)

        result = []
        for proc_block in balanced_blocks:
            result.append(IRInstruction('PROCESSOR_BEGIN', []))
            result.extend(proc_block)
            result.append(IRInstruction('PROCESSOR_END', []))
        return result

    def print_ir(self):
        for inst in self.ir:
            print(inst)


class MIMDInstruction:
    def __init__(self, op, args):
        self.op = op
        self.args = args

    def __str__(self):
        return f"{self.op} {', '.join(map(str, self.args))}"

class CodeGenerator:
    def __init__(self, ir):
        self.ir = ir
        self.code = [[] for _ in range(4)]  # 4 processors
        self.current_processor = 0
        self.in_parallel = False

    def generate(self):
        for inst in self.ir:
            if inst.op == 'PARALLEL_BEGIN':
                self.in_parallel = True
                self.current_processor = 0
            elif inst.op == 'PARALLEL_END':
                self.in_parallel = False
                self.current_processor = 0
            elif self.in_parallel:
                self.emit(inst)
                self.current_processor = (self.current_processor + 1) % 4
            else:
                self.emit(inst)

        return self.code

    def emit(self, inst):
        formatted_inst = self.format_instruction(inst)
        if self.in_parallel:
            self.code[self.current_processor].append(formatted_inst)
        else:
            for p in range(4):
                self.code[p].append(formatted_inst)

    def format_instruction(self, inst):
        if not isinstance(inst.args, list):
            return f"{inst.op} {inst.args}"

        if inst.op in ['LOAD', 'STORE', 'ALLOC']:
            return f"{inst.op} {', '.join(map(str, inst.args))}"
        elif inst.op in ['ADD', 'SUB', 'MUL', 'DIV']:
            return f"{inst.op} {', '.join(map(str, inst.args))}"
        elif inst.op == 'RETURN':
            return f"RET {inst.args[0]}"
        elif inst.op in ['FUNCTION_BEGIN', 'FUNCTION_END']:
            return f"{inst.op} {inst.args[0]}"
        elif inst.op in ['PARALLEL_BEGIN', 'PARALLEL_END']:
            return inst.op
        else:
            return f"{inst.op} {' '.join(map(str, inst.args))}"


    def print_code(self):
        for i, processor_code in enumerate(self.code):
            print(f"Processor {i}:")
            for inst in processor_code:
                print(f"  {inst}")
            print()

class Compiler:
    def __init__(self, source_code):
        self.source_code = source_code
        self.lexer = Lexer(source_code)
        self.parser = None
        self.semantic_analyzer = None
        self.ir_generator = None
        self.ir_optimizer = None
        self.code_generator = None

    def compile(self):
        # Lexical Analysis
        tokens = self.lexer.tokenize()
        print("Tokens:")
        for token in tokens:
            print(token)

        # Syntax Analysis
        self.parser = Parser(tokens)
        ast = self.parser.parse()
        print("\nAbstract Syntax Tree:")
        self.print_ast(ast)

        # Semantic Analysis
        self.semantic_analyzer = SemanticAnalyzer(ast)
        self.semantic_analyzer.analyze()
        print("\nSemantic Analysis completed")

        # IR Generation
        self.ir_generator = IRGenerator(ast)
        ir = self.ir_generator.generate()
        print("\nIntermediate Representation:")
        for inst in ir:
            print(inst)

        # IR Optimization
        self.ir_optimizer = IROptimizer(ir)
        optimized_ir = self.ir_optimizer.optimize()
        print("\nOptimized Intermediate Representation:")
        for inst in optimized_ir:
            print(inst)

        # Code Generation
        self.code_generator = CodeGenerator(optimized_ir)
        mimd_code = self.code_generator.generate()
        print("\nGenerated MIMD Code:")
        self.code_generator.print_code()

        return mimd_code

    def print_ast(self, node, level=0):
        print('  ' * level + f"{node.type}: {node.value}")
        for child in node.children:
            self.print_ast(child, level + 1)

# Example usage
source_code = """
int main() {
    int a = 5;
    int b = 10;
    int c;
    parallel {
        a = a + 1;
        b = b * 2;
    }
    c = a + b;
    return c;
}
"""

compiler = Compiler(source_code)
mimd_code = compiler.compile()

Tokens:
Token(TokenType.KEYWORD, int, line=2, col=4)
Token(TokenType.IDENTIFIER, main, line=2, col=9)
Token(TokenType.PUNCTUATOR, LPAREN, line=2, col=9)
Token(TokenType.PUNCTUATOR, RPAREN, line=2, col=10)
Token(TokenType.PUNCTUATOR, LBRACE, line=2, col=12)
Token(TokenType.KEYWORD, int, line=3, col=8)
Token(TokenType.IDENTIFIER, a, line=3, col=10)
Token(TokenType.OPERATOR, ASSIGN, line=3, col=11)
Token(TokenType.NUMBER, 5, line=3, col=14)
Token(TokenType.PUNCTUATOR, SEMI, line=3, col=14)
Token(TokenType.KEYWORD, int, line=4, col=8)
Token(TokenType.IDENTIFIER, b, line=4, col=10)
Token(TokenType.OPERATOR, ASSIGN, line=4, col=11)
Token(TokenType.NUMBER, 10, line=4, col=15)
Token(TokenType.PUNCTUATOR, SEMI, line=4, col=15)
Token(TokenType.KEYWORD, int, line=5, col=8)
Token(TokenType.IDENTIFIER, c, line=5, col=10)
Token(TokenType.PUNCTUATOR, SEMI, line=5, col=10)
Token(TokenType.KEYWORD, parallel, line=6, col=13)
Token(TokenType.PUNCTUATOR, LBRACE, line=6, col=14)
Token(TokenType.IDENTIFIER,